In [1]:
from sklearn.model_selection import train_test_split
from database import filter_hyposmia
import pandas as pd
from scipy.spatial import KDTree
import numpy as np

# Load the datasets
data_large = pd.read_csv('data/processed/remote_processed.csv', low_memory=False) #target = 0
data_small = pd.read_csv('data/processed/clinical_processed_imputed.csv', low_memory=False) #target = 1

print(len(data_large))
print(len(data_small))

# Filter hyposmia
data_large = filter_hyposmia(data_large, '2023')
data_small = filter_hyposmia(data_small, '2023')

print(len(data_large))
print(len(data_small))

age_var = 'AGE'
sex_var = 'BIRTHSEX'

# Ensure both datasets have age_var and sex_var columns
assert age_var in data_large.columns and sex_var in data_large.columns
assert age_var in data_small.columns and sex_var in data_small.columns

21236
366
Using 2023 article cutoff
Using 2023 article cutoff
1504
194


In [2]:
# Create a new DataFrame to store the balanced dataset
balanced_data = pd.DataFrame()

# Set to keep track of used indices
used_indices = set()

# Loop through unique combinations of age and sex in the smaller dataset
for (age, sex), group in data_small.groupby([age_var, sex_var]):
    # Find all matching rows in the larger dataset by sex
    matching_rows = data_large[data_large[sex_var] == sex]
    
    if len(matching_rows) > 0:
        # Build a KDTree for the current sex group
        kd_tree = KDTree(matching_rows[[age_var]].values)
        
        # Find the closest age values using KDTree
        _, indices = kd_tree.query([[age]], k=len(matching_rows))
        closest_indices = matching_rows.index[indices.flatten()]
        
        # Filter out already used indices
        closest_indices = [idx for idx in closest_indices if idx not in used_indices]
        
        # If there are not enough unique closest indices, limit to available ones
        closest_indices = closest_indices[:len(group)]
        
        # Update the set of used indices
        used_indices.update(closest_indices)
        
        # Get the sampled rows
        sampled_rows = data_large.loc[closest_indices]
        
        # Append the sampled rows to the balanced dataset
        balanced_data = pd.concat([balanced_data, sampled_rows])
    else:
        # If no matching rows are found, you might want to log this or handle it differently
        print(f"No matching rows found for sex {sex}")

# Reset the index of the balanced dataset
balanced_data.reset_index(drop=True, inplace=True)

print(f"Balanced dataset created with {len(balanced_data)*2} instances.")


Balanced dataset created with 388 instances.


In [3]:
print(balanced_data.shape)
print(data_small.shape)

(194, 162)
(194, 165)


In [4]:
#print mean and std of age and sex
print(balanced_data['AGE'].mean(), data_small['AGE'].mean())
print(balanced_data['AGE'].std(), data_small['AGE'].std())
print(balanced_data['BIRTHSEX'].mean(), data_small['BIRTHSEX'].mean())
print(balanced_data['BIRTHSEX'].std(), data_small['BIRTHSEX'].std())


68.12886597938144 68.04278350515463
5.635646332778791 5.687680941451203
0.6701030927835051 0.6701030927835051
0.47139193036274024 0.47139193036274024


In [5]:
#check for dupicate in PATNO in both datasets
print(balanced_data['PATNO'].duplicated().sum())
print(data_small['PATNO'].duplicated().sum())
#print which patno is duplicate on data_small
print(data_small[data_small['PATNO'].duplicated()])

0
0
Empty DataFrame
Columns: [PATNO, EVENT_ID, AGE, REC_ID_x, PAG_NAME_x, INFODT_x, AFICBERB, ASHKJEW, BASQUE, BIRTHDT, BIRTHSEX, CHLDBEAR, HOWLIVE, GAYLES, HETERO, BISEXUAL, PANSEXUAL, ASEXUAL, OTHSEXUALITY, HANDED, HISPLAT, RAASIAN, RABLACK, RAHAWOPI, RAINDALS, RANOS, RAWHITE, RAUNKNOWN, ORIG_ENTRY_x, LAST_UPDATE_x, EDUCYRS, REC_ID, PAG_NAME, INFODT, SCENT_01_CORRECT, SCENT_01_RESPONSE, SCENT_02_CORRECT, SCENT_02_RESPONSE, SCENT_03_CORRECT, SCENT_03_RESPONSE, SCENT_04_CORRECT, SCENT_04_RESPONSE, SCENT_05_CORRECT, SCENT_05_RESPONSE, SCENT_06_CORRECT, SCENT_06_RESPONSE, SCENT_07_CORRECT, SCENT_07_RESPONSE, SCENT_08_CORRECT, SCENT_08_RESPONSE, SCENT_09_CORRECT, SCENT_09_RESPONSE, SCENT_10_CORRECT, SCENT_10_RESPONSE, SCENT_11_CORRECT, SCENT_11_RESPONSE, SCENT_12_CORRECT, SCENT_12_RESPONSE, SCENT_13_CORRECT, SCENT_13_RESPONSE, SCENT_14_CORRECT, SCENT_14_RESPONSE, SCENT_15_CORRECT, SCENT_15_RESPONSE, SCENT_16_CORRECT, SCENT_16_RESPONSE, SCENT_17_CORRECT, SCENT_17_RESPONSE, SCENT_18_CORRECT

In [6]:

remote = pd.read_csv('data/processed/remote_processed.csv')
from tqdm import tqdm

#check, for each PATNO in data_balanced, if it is in remote, and if it has the equal values in all columns
for i in tqdm(range(len(balanced_data))):
    patno = balanced_data['PATNO'].iloc[i]
    if patno in remote['PATNO'].values:
        #check if values are equal for all columns
        for col in balanced_data.columns:
            if col != 'PATNO':
                #ignore if both are nan
                if pd.isnull(balanced_data[col].iloc[i]) and pd.isnull(remote[remote['PATNO'] == patno][col].values[0]):
                    continue
                if balanced_data[col].iloc[i] != remote[remote['PATNO'] == patno][col].values[0]:
                    print(f"Row {i} has different value in column {col}")
                    #print values
                    print(balanced_data[col].iloc[i], remote[remote['PATNO'] == patno][col].values[0])
    else:
        print(f"Row {i} has no matching PATNO {patno} in remote")

/tmp/ipykernel_71230/3628960552.py:1: DtypeWarning: Columns (14,16,19,22,54) have mixed types. Specify dtype option on import or set low_memory=False.
  remote = pd.read_csv('data/processed/remote_processed.csv')
100%|██████████| 194/194 [00:13<00:00, 14.06it/s]


In [7]:
# Load the datasets
data_large = balanced_data
data_small = pd.read_csv('data/processed/clinical_processed_imputed.csv')

#filter for HYPOSMIA =1
data_large = data_large[data_large['HYPOSMIA'] == 1]
data_small = data_small[data_small['HYPOSMIA'] == 1]

# Add a target column to each dataset
data_large['target'] = 0
data_small['target'] = 1

# Combine the datasets
combined_data = pd.concat([data_large, data_small])

# Reset index of combined data
combined_data.reset_index(drop=True, inplace=True)

# Define the columns for stratification
age_var = 'AGE'
sex_var = 'BIRTHSEX'
target_var = 'target'

# Ensure both datasets have age_var, sex_var, and target_var columns
assert age_var in combined_data.columns and sex_var in combined_data.columns and target_var in combined_data.columns

# Calculate percentiles for the age variable
percentiles = np.percentile(combined_data[age_var], [20, 40, 60, 80])
labels = ['0-20th', '21-40th', '41-60th', '61-80th', '81-100th']
combined_data['age_binned'] = pd.cut(combined_data[age_var], bins=[combined_data[age_var].min()-1, *percentiles, combined_data[age_var].max()+1], labels=labels, include_lowest=True)

# Create a stratification key
combined_data['stratify_key'] = combined_data.apply(lambda row: f"{row['age_binned']}_{row[sex_var]}_{row[target_var]}", axis=1)

# Split the data into train and test sets
train, test = train_test_split(combined_data, test_size=0.2, stratify=combined_data['stratify_key'], random_state=42)

# Drop the stratification key and binned age column
train = train.drop(columns=['stratify_key', 'age_binned'])
test = test.drop(columns=['stratify_key', 'age_binned'])

#print mean, std and count of sex, age and target in train and test
print(train['BIRTHSEX'].mean(), test['BIRTHSEX'].mean())
print(train['BIRTHSEX'].std(), test['BIRTHSEX'].std())
print(train['BIRTHSEX'].count(), test['BIRTHSEX'].count())
print(train['AGE'].mean(), test['AGE'].mean())
print(train['AGE'].std(), test['AGE'].std())
print(train['AGE'].count(), test['AGE'].count())
print(train['target'].mean(), test['target'].mean())
print(train['target'].std(), test['target'].std())
print(train['target'].count(), test['target'].count())

#switch random patients from train to test until target is balanced
while abs(train['target'].mean() - test['target'].mean()) > 0.01:
    #get random patient from train
    random_patient = train.sample()
    #switch to test
    test = pd.concat([test, random_patient])
    train = train.drop(random_patient.index)
    #now switch from test to train
    random_patient = test.sample()
    train = pd.concat([train, random_patient])
    test = test.drop(random_patient.index)
    #print mean of target in train and test
    print(train['target'].mean(), test['target'].mean())
print ("Balanced target")
#print mean, std and count of sex, age and target in train and test
print(train['BIRTHSEX'].mean(), test['BIRTHSEX'].mean())
print(train['BIRTHSEX'].std(), test['BIRTHSEX'].std())
print(train['BIRTHSEX'].count(), test['BIRTHSEX'].count())
print(train['AGE'].mean(), test['AGE'].mean())
print(train['AGE'].std(), test['AGE'].std())
print(train['AGE'].count(), test['AGE'].count())
print(train['target'].mean(), test['target'].mean())
print(train['target'].std(), test['target'].std())
print(train['target'].count(), test['target'].count())


#print shapes
print(train.shape)
print(test.shape)

0.6709677419354839 0.6666666666666666
0.470621394900324 0.47445571459117775
310 78
68.07322580645162 68.13589743589743
5.789385380850459 5.118085436449702
310 78
0.4967741935483871 0.5128205128205128
0.5007979852052057 0.5030708231255845
310 78
0.4967741935483871 0.5128205128205128
0.5 0.5
Balanced target
0.6709677419354839 0.6666666666666666
0.470621394900324 0.47445571459117764
310 78
68.03258064516129 68.2974358974359
5.736738170751184 5.34568582843745
310 78
0.5 0.5
0.5008084079652348 0.5032362797401965
310 78
(310, 222)
(78, 222)


In [8]:
#check if there is any duplicate in PATNO
print(train['PATNO'].duplicated().sum())
print(test['PATNO'].duplicated().sum())

print(train.shape)
print(test.shape)

train_columns = train.columns
test_columns = test.columns

if train_columns.equals(test_columns):
    print("Columns are the same")

for column in train_columns:
    print(column)

0
0
(310, 222)
(78, 222)
Columns are the same
PATNO
EVENT_ID
PAG_NAME
INFODT
HIQ1
HIQ2
HIQ3
HIQ4
HIQ5
HIQ6
ORIG_ENTRY
LAST_UPDATE
UPSITSNTDT
UPSITENTDT
SCRINTDT
SCRINTST
SCRTRDT
SCRSITE
SCRVISTYPE
SCRSCHEDDT
SCRVISST
SCRRSNUM
BLINTDT
BLINTST
BLTRDT
BLSITE
BLSCHEDDT
BLVISST
BLRSNUM
COHORT
TRACK
CONSENTDT
CONSENTST
ENROLLST
BIRTHSEX
RAASIAN
RABLACK
RADEC
RAHAWOPI
RAINDALS
RAUNK
RAWHITE
HISPLAT
COUNTRY
SCREENINGID
SCREENDT
SCREENTM
CAMPAIGN
REQAGE
REQLOC
DIAGPD
PDMED
DIAGAGE2
RTRNLTR
PARTNERID
USQ1
USQ2
USQ3
USQ4
USQ5
USQ6
USQ7
USQ8
USQ8A
USQ8B
USQ8C
USQ9
USQ10
REC_ID
SCENT_01_CORRECT
SCENT_01_RESPONSE
SCENT_02_CORRECT
SCENT_02_RESPONSE
SCENT_03_CORRECT
SCENT_03_RESPONSE
SCENT_04_CORRECT
SCENT_04_RESPONSE
SCENT_05_CORRECT
SCENT_05_RESPONSE
SCENT_06_CORRECT
SCENT_06_RESPONSE
SCENT_07_CORRECT
SCENT_07_RESPONSE
SCENT_08_CORRECT
SCENT_08_RESPONSE
SCENT_09_CORRECT
SCENT_09_RESPONSE
SCENT_10_CORRECT
SCENT_10_RESPONSE
SCENT_11_CORRECT
SCENT_11_RESPONSE
SCENT_12_CORRECT
SCENT_12_RESPONSE
SCENT_13

In [9]:
#sum all correct columns (that contain "CORRECT")

#print mean and std of total_correct
print(train['TOTAL_CORRECT'].mean(), test['TOTAL_CORRECT'].mean())
#MIN AND MAX
print(train['TOTAL_CORRECT'].min(), test['TOTAL_CORRECT'].min())
print(train['TOTAL_CORRECT'].max(), test['TOTAL_CORRECT'].max())
#print mean and std of TOTAL_CORRECT per target
print(train.groupby('target')['TOTAL_CORRECT'].mean())
print(test.groupby('target')['TOTAL_CORRECT'].mean())
#print mean and std of TOTAL_CORRECT
print(train.groupby('HYPOSMIA')['TOTAL_CORRECT'].min())
print(test.groupby('HYPOSMIA')['TOTAL_CORRECT'].min())
#now per HYPOSMIA
print(train.groupby('HYPOSMIA')['TOTAL_CORRECT'].max())
print(test.groupby('HYPOSMIA')['TOTAL_CORRECT'].max())
 


18.233064516129033 17.814102564102566
4.0 7.0
30.0 30.0
target
0    18.503226
1    17.962903
Name: TOTAL_CORRECT, dtype: float64
target
0    18.615385
1    17.012821
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    4.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    7.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    30.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    30.0
Name: TOTAL_CORRECT, dtype: float64


In [10]:

# Save the train and test sets to new CSV files
train.to_csv('data/processed/train_data_hyposmia2023.csv', index=False)
test.to_csv('data/processed/test_data_hyposmia2023.csv', index=False)

print(f"Train set created with {len(train)} instances.")
print(f"Test set created with {len(test)} instances.")

Train set created with 310 instances.
Test set created with 78 instances.


clinical data without imputation

In [11]:
# Load the datasets
data_large = pd.read_csv('data/processed/remote_processed.csv', low_memory=False) #target = 0
data_small = pd.read_csv('data/processed/clinical_processed.csv', low_memory=False) #target = 1

print(len(data_large))
print(len(data_small))

# Filter hyposmia
data_large = filter_hyposmia(data_large, '2023')
data_small = filter_hyposmia(data_small, '2023')

print(len(data_large))
print(len(data_small))

age_var = 'AGE'
sex_var = 'BIRTHSEX'

# Ensure both datasets have age_var and sex_var columns
assert age_var in data_large.columns and sex_var in data_large.columns
assert age_var in data_small.columns and sex_var in data_small.columns

21236
248
Using 2023 article cutoff
Using 2023 article cutoff
1504
125


In [12]:
# Create a new DataFrame to store the balanced dataset
balanced_data = pd.DataFrame()

# Set to keep track of used indices
used_indices = set()

# Loop through unique combinations of age and sex in the smaller dataset
for (age, sex), group in data_small.groupby([age_var, sex_var]):
    # Find all matching rows in the larger dataset by sex
    matching_rows = data_large[data_large[sex_var] == sex]
    
    if len(matching_rows) > 0:
        # Build a KDTree for the current sex group
        kd_tree = KDTree(matching_rows[[age_var]].values)
        
        # Find the closest age values using KDTree
        _, indices = kd_tree.query([[age]], k=len(matching_rows))
        closest_indices = matching_rows.index[indices.flatten()]
        
        # Filter out already used indices
        closest_indices = [idx for idx in closest_indices if idx not in used_indices]
        
        # If there are not enough unique closest indices, limit to available ones
        closest_indices = closest_indices[:len(group)]
        
        # Update the set of used indices
        used_indices.update(closest_indices)
        
        # Get the sampled rows
        sampled_rows = data_large.loc[closest_indices]
        
        # Append the sampled rows to the balanced dataset
        balanced_data = pd.concat([balanced_data, sampled_rows])
    else:
        # If no matching rows are found, you might want to log this or handle it differently
        print(f"No matching rows found for sex {sex}")

# Reset the index of the balanced dataset
balanced_data.reset_index(drop=True, inplace=True)

print(f"Balanced dataset created with {len(balanced_data)*2} instances.")


Balanced dataset created with 250 instances.


In [13]:
print(balanced_data.shape)
print(data_small.shape)

(125, 162)
(125, 165)


In [14]:
#print mean and std of age and sex
print(balanced_data['AGE'].mean(), data_small['AGE'].mean())
print(balanced_data['AGE'].std(), data_small['AGE'].std())
print(balanced_data['BIRTHSEX'].mean(), data_small['BIRTHSEX'].mean())
print(balanced_data['BIRTHSEX'].std(), data_small['BIRTHSEX'].std())


68.472 68.4096
5.660525451501545 5.680064515762634
0.672 0.672
0.4713741066150078 0.47137410661500784


In [15]:
#check for dupicate in PATNO in both datasets
print(balanced_data['PATNO'].duplicated().sum())
print(data_small['PATNO'].duplicated().sum())
#print which patno is duplicate on data_small
print(data_small[data_small['PATNO'].duplicated()])

0
0
Empty DataFrame
Columns: [PATNO, EVENT_ID, AGE, REC_ID_x, PAG_NAME_x, INFODT_x, AFICBERB, ASHKJEW, BASQUE, BIRTHDT, BIRTHSEX, CHLDBEAR, HOWLIVE, GAYLES, HETERO, BISEXUAL, PANSEXUAL, ASEXUAL, OTHSEXUALITY, HANDED, HISPLAT, RAASIAN, RABLACK, RAHAWOPI, RAINDALS, RANOS, RAWHITE, RAUNKNOWN, ORIG_ENTRY_x, LAST_UPDATE_x, EDUCYRS, REC_ID, PAG_NAME, INFODT, SCENT_01_CORRECT, SCENT_01_RESPONSE, SCENT_02_CORRECT, SCENT_02_RESPONSE, SCENT_03_CORRECT, SCENT_03_RESPONSE, SCENT_04_CORRECT, SCENT_04_RESPONSE, SCENT_05_CORRECT, SCENT_05_RESPONSE, SCENT_06_CORRECT, SCENT_06_RESPONSE, SCENT_07_CORRECT, SCENT_07_RESPONSE, SCENT_08_CORRECT, SCENT_08_RESPONSE, SCENT_09_CORRECT, SCENT_09_RESPONSE, SCENT_10_CORRECT, SCENT_10_RESPONSE, SCENT_11_CORRECT, SCENT_11_RESPONSE, SCENT_12_CORRECT, SCENT_12_RESPONSE, SCENT_13_CORRECT, SCENT_13_RESPONSE, SCENT_14_CORRECT, SCENT_14_RESPONSE, SCENT_15_CORRECT, SCENT_15_RESPONSE, SCENT_16_CORRECT, SCENT_16_RESPONSE, SCENT_17_CORRECT, SCENT_17_RESPONSE, SCENT_18_CORRECT

In [16]:

remote = pd.read_csv('data/processed/remote_processed.csv')
from tqdm import tqdm

#check, for each PATNO in data_balanced, if it is in remote, and if it has the equal values in all columns
for i in tqdm(range(len(balanced_data))):
    patno = balanced_data['PATNO'].iloc[i]
    if patno in remote['PATNO'].values:
        #check if values are equal for all columns
        for col in balanced_data.columns:
            if col != 'PATNO':
                #ignore if both are nan
                if pd.isnull(balanced_data[col].iloc[i]) and pd.isnull(remote[remote['PATNO'] == patno][col].values[0]):
                    continue
                if balanced_data[col].iloc[i] != remote[remote['PATNO'] == patno][col].values[0]:
                    print(f"Row {i} has different value in column {col}")
                    #print values
                    print(balanced_data[col].iloc[i], remote[remote['PATNO'] == patno][col].values[0])
    else:
        print(f"Row {i} has no matching PATNO {patno} in remote")

/tmp/ipykernel_71230/3628960552.py:1: DtypeWarning: Columns (14,16,19,22,54) have mixed types. Specify dtype option on import or set low_memory=False.
  remote = pd.read_csv('data/processed/remote_processed.csv')
100%|██████████| 125/125 [00:08<00:00, 14.16it/s]


In [17]:
# Load the datasets
data_large = balanced_data
data_small = pd.read_csv('data/processed/clinical_processed.csv')

#filter for HYPOSMIA =1
data_large = data_large[data_large['HYPOSMIA'] == 1]
data_small = data_small[data_small['HYPOSMIA'] == 1]

# Add a target column to each dataset
data_large['target'] = 0
data_small['target'] = 1

# Combine the datasets
combined_data = pd.concat([data_large, data_small])

# Reset index of combined data
combined_data.reset_index(drop=True, inplace=True)

# Define the columns for stratification
age_var = 'AGE'
sex_var = 'BIRTHSEX'
target_var = 'target'

# Ensure both datasets have age_var, sex_var, and target_var columns
assert age_var in combined_data.columns and sex_var in combined_data.columns and target_var in combined_data.columns

# Calculate percentiles for the age variable
percentiles = np.percentile(combined_data[age_var], [20, 40, 60, 80])
labels = ['0-20th', '21-40th', '41-60th', '61-80th', '81-100th']
combined_data['age_binned'] = pd.cut(combined_data[age_var], bins=[combined_data[age_var].min()-1, *percentiles, combined_data[age_var].max()+1], labels=labels, include_lowest=True)

# Create a stratification key
combined_data['stratify_key'] = combined_data.apply(lambda row: f"{row['age_binned']}_{row[sex_var]}_{row[target_var]}", axis=1)

# Split the data into train and test sets
train, test = train_test_split(combined_data, test_size=0.2, stratify=combined_data['stratify_key'], random_state=42)

# Drop the stratification key and binned age column
train = train.drop(columns=['stratify_key', 'age_binned'])
test = test.drop(columns=['stratify_key', 'age_binned'])

#print mean, std and count of sex, age and target in train and test
print(train['BIRTHSEX'].mean(), test['BIRTHSEX'].mean())
print(train['BIRTHSEX'].std(), test['BIRTHSEX'].std())
print(train['BIRTHSEX'].count(), test['BIRTHSEX'].count())
print(train['AGE'].mean(), test['AGE'].mean())
print(train['AGE'].std(), test['AGE'].std())
print(train['AGE'].count(), test['AGE'].count())
print(train['target'].mean(), test['target'].mean())
print(train['target'].std(), test['target'].std())
print(train['target'].count(), test['target'].count())

#switch random patients from train to test until target is balanced
while abs(train['target'].mean() - test['target'].mean()) > 0.01:
    #get random patient from train
    random_patient = train.sample()
    #switch to test
    test = pd.concat([test, random_patient])
    train = train.drop(random_patient.index)
    #now switch from test to train
    random_patient = test.sample()
    train = pd.concat([train, random_patient])
    test = test.drop(random_patient.index)
    #print mean of target in train and test
    print(train['target'].mean(), test['target'].mean())
print ("Balanced target")
#print mean, std and count of sex, age and target in train and test
print(train['BIRTHSEX'].mean(), test['BIRTHSEX'].mean())
print(train['BIRTHSEX'].std(), test['BIRTHSEX'].std())
print(train['BIRTHSEX'].count(), test['BIRTHSEX'].count())
print(train['AGE'].mean(), test['AGE'].mean())
print(train['AGE'].std(), test['AGE'].std())
print(train['AGE'].count(), test['AGE'].count())
print(train['target'].mean(), test['target'].mean())
print(train['target'].std(), test['target'].std())
print(train['target'].count(), test['target'].count())


#print shapes
print(train.shape)
print(test.shape)

0.67 0.68
0.4713926763075365 0.47121207149916117
200 50
68.5315 68.078
5.753037890964597 5.30576467396644
200 50
0.5 0.5
0.5012547071170855 0.5050762722761054
200 50
Balanced target
0.67 0.68
0.4713926763075365 0.47121207149916117
200 50
68.5315 68.078
5.753037890964597 5.30576467396644
200 50
0.5 0.5
0.5012547071170855 0.5050762722761054
200 50
(200, 222)
(50, 222)


In [18]:
#check if there is any duplicate in PATNO
print(train['PATNO'].duplicated().sum())
print(test['PATNO'].duplicated().sum())

print(train.shape)
print(test.shape)

train_columns = train.columns
test_columns = test.columns

if train_columns.equals(test_columns):
    print("Columns are the same")

for column in train_columns:
    print(column)

0
0
(200, 222)
(50, 222)
Columns are the same
PATNO
EVENT_ID
PAG_NAME
INFODT
HIQ1
HIQ2
HIQ3
HIQ4
HIQ5
HIQ6
ORIG_ENTRY
LAST_UPDATE
UPSITSNTDT
UPSITENTDT
SCRINTDT
SCRINTST
SCRTRDT
SCRSITE
SCRVISTYPE
SCRSCHEDDT
SCRVISST
SCRRSNUM
BLINTDT
BLINTST
BLTRDT
BLSITE
BLSCHEDDT
BLVISST
BLRSNUM
COHORT
TRACK
CONSENTDT
CONSENTST
ENROLLST
BIRTHSEX
RAASIAN
RABLACK
RADEC
RAHAWOPI
RAINDALS
RAUNK
RAWHITE
HISPLAT
COUNTRY
SCREENINGID
SCREENDT
SCREENTM
CAMPAIGN
REQAGE
REQLOC
DIAGPD
PDMED
DIAGAGE2
RTRNLTR
PARTNERID
USQ1
USQ2
USQ3
USQ4
USQ5
USQ6
USQ7
USQ8
USQ8A
USQ8B
USQ8C
USQ9
USQ10
REC_ID
SCENT_01_CORRECT
SCENT_01_RESPONSE
SCENT_02_CORRECT
SCENT_02_RESPONSE
SCENT_03_CORRECT
SCENT_03_RESPONSE
SCENT_04_CORRECT
SCENT_04_RESPONSE
SCENT_05_CORRECT
SCENT_05_RESPONSE
SCENT_06_CORRECT
SCENT_06_RESPONSE
SCENT_07_CORRECT
SCENT_07_RESPONSE
SCENT_08_CORRECT
SCENT_08_RESPONSE
SCENT_09_CORRECT
SCENT_09_RESPONSE
SCENT_10_CORRECT
SCENT_10_RESPONSE
SCENT_11_CORRECT
SCENT_11_RESPONSE
SCENT_12_CORRECT
SCENT_12_RESPONSE
SCENT_13

In [19]:
#sum all correct columns (that contain "CORRECT")

#print mean and std of total_correct
print(train['TOTAL_CORRECT'].mean(), test['TOTAL_CORRECT'].mean())
#MIN AND MAX
print(train['TOTAL_CORRECT'].min(), test['TOTAL_CORRECT'].min())
print(train['TOTAL_CORRECT'].max(), test['TOTAL_CORRECT'].max())
#print mean and std of TOTAL_CORRECT per target
print(train.groupby('target')['TOTAL_CORRECT'].mean())
print(test.groupby('target')['TOTAL_CORRECT'].mean())
#print mean and std of TOTAL_CORRECT
print(train.groupby('HYPOSMIA')['TOTAL_CORRECT'].min())
print(test.groupby('HYPOSMIA')['TOTAL_CORRECT'].min())
#now per HYPOSMIA
print(train.groupby('HYPOSMIA')['TOTAL_CORRECT'].max())
print(test.groupby('HYPOSMIA')['TOTAL_CORRECT'].max())
 


17.73 16.7
6.0 7.0
30.0 27.0
target
0    18.19
1    17.27
Name: TOTAL_CORRECT, dtype: float64
target
0    18.44
1    14.96
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    6.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    7.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    30.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
1.0    27.0
Name: TOTAL_CORRECT, dtype: float64


In [20]:

# Save the train and test sets to new CSV files
train.to_csv('data/processed/train_data_without_imput.csv', index=False)
test.to_csv('data/processed/test_data_without_imput.csv', index=False)

print(f"Train set created with {len(train)} instances.")
print(f"Test set created with {len(test)} instances.")

Train set created with 200 instances.
Test set created with 50 instances.


NOT FILTERING FOR HYPOSMIA

In [21]:
from sklearn.model_selection import train_test_split
from database import filter_hyposmia
import pandas as pd
from scipy.spatial import KDTree
import numpy as np

# Load the datasets
data_large = pd.read_csv('data/processed/remote_processed.csv', low_memory=False) #target = 0
data_small = pd.read_csv('data/processed/clinical_processed_imputed.csv', low_memory=False) #target = 1

print(len(data_large))
print(len(data_small))

# Filter hyposmia
#data_large = filter_hyposmia(data_large, '2023')
#data_small = filter_hyposmia(data_small, '2023')

age_var = 'AGE'
sex_var = 'BIRTHSEX'

# Ensure both datasets have age_var and sex_var columns
assert age_var in data_large.columns and sex_var in data_large.columns
assert age_var in data_small.columns and sex_var in data_small.columns

21236
366


In [22]:
# Create a new DataFrame to store the balanced dataset
balanced_data = pd.DataFrame()

# Set to keep track of used indices
used_indices = set()

# Loop through unique combinations of age and sex in the smaller dataset
for (age, sex), group in data_small.groupby([age_var, sex_var]):
    # Find all matching rows in the larger dataset by sex
    matching_rows = data_large[data_large[sex_var] == sex]
    
    if len(matching_rows) > 0:
        # Build a KDTree for the current sex group
        kd_tree = KDTree(matching_rows[[age_var]].values)
        
        # Find the closest age values using KDTree
        _, indices = kd_tree.query([[age]], k=len(matching_rows))
        closest_indices = matching_rows.index[indices.flatten()]
        
        # Filter out already used indices
        closest_indices = [idx for idx in closest_indices if idx not in used_indices]
        
        # If there are not enough unique closest indices, limit to available ones
        closest_indices = closest_indices[:len(group)]
        
        # Update the set of used indices
        used_indices.update(closest_indices)
        
        # Get the sampled rows
        sampled_rows = data_large.loc[closest_indices]
        
        # Append the sampled rows to the balanced dataset
        balanced_data = pd.concat([balanced_data, sampled_rows])
    else:
        # If no matching rows are found, you might want to log this or handle it differently
        print(f"No matching rows found for sex {sex}")

# Reset the index of the balanced dataset
balanced_data.reset_index(drop=True, inplace=True)

print(f"Balanced dataset created with {len(balanced_data)*2} instances.")


Balanced dataset created with 732 instances.


In [23]:
print(balanced_data.shape)
print(data_small.shape)

(366, 162)
(366, 165)


In [24]:
#print mean and std of age and sex
print(balanced_data['AGE'].mean(), data_small['AGE'].mean())
print(balanced_data['AGE'].std(), data_small['AGE'].std())
print(balanced_data['BIRTHSEX'].mean(), data_small['BIRTHSEX'].mean())
print(balanced_data['BIRTHSEX'].std(), data_small['BIRTHSEX'].std())


68.74043715846994 68.7448087431694
5.858220200535136 5.861241426456602
0.6338797814207651 0.6338797814207651
0.482402352133065 0.482402352133065


In [25]:
#check for dupicate in PATNO in both datasets
print(balanced_data['PATNO'].duplicated().sum())
print(data_small['PATNO'].duplicated().sum())
#print which patno is duplicate on data_small
print(data_small[data_small['PATNO'].duplicated()])

#drop duplicates
data_small = data_small.drop_duplicates(subset='PATNO')

0
2
      PATNO EVENT_ID   AGE  REC_ID_x PAG_NAME_x INFODT_x  AFICBERB  ASHKJEW  \
85   130769     BLSC  75.4   IA87748     SCREEN  06/2022       0.0      0.0   
318  262152     BLSC  64.6  IA443252     SCREEN  03/2024       0.0      0.0   

     BASQUE  BIRTHDT  ...  ENRLPRKN  ENRLSRDC  ENRLHPSM  ENRLRBD  ENRLLRRK2  \
85      0.0  03/1947  ...       0.0       1.0       0.0      0.0        0.0   
318     0.0  09/1959  ...       0.0       1.0       0.0      0.0        0.0   

     ENRLSNCA  ENRLGBA  HYPOSMIA  HYPOSMIA_PPMI  PD_Duration  
85        0.0      0.0       0.0            0.0       -121.0  
318       0.0      0.0       0.0            0.0         30.0  

[2 rows x 165 columns]


In [26]:

remote = pd.read_csv('data/processed/remote_processed.csv')
from tqdm import tqdm

#check, for each PATNO in data_balanced, if it is in remote, and if it has the equal values in all columns
for i in tqdm(range(len(balanced_data))):
    patno = balanced_data['PATNO'].iloc[i]
    if patno in remote['PATNO'].values:
        #check if values are equal for all columns
        for col in balanced_data.columns:
            if col != 'PATNO':
                #ignore if both are nan
                if pd.isnull(balanced_data[col].iloc[i]) and pd.isnull(remote[remote['PATNO'] == patno][col].values[0]):
                    continue
                if balanced_data[col].iloc[i] != remote[remote['PATNO'] == patno][col].values[0]:
                    print(f"Row {i} has different value in column {col}")
                    #print values
                    print(balanced_data[col].iloc[i], remote[remote['PATNO'] == patno][col].values[0])
    else:
        print(f"Row {i} has no matching PATNO {patno} in remote")

/tmp/ipykernel_71230/3628960552.py:1: DtypeWarning: Columns (14,16,19,22,54) have mixed types. Specify dtype option on import or set low_memory=False.
  remote = pd.read_csv('data/processed/remote_processed.csv')
100%|██████████| 366/366 [00:26<00:00, 13.96it/s]


In [27]:
# Load the datasets
data_large = balanced_data
data_small = pd.read_csv('data/processed/clinical_processed_imputed.csv')

#drop duplicates
data_small = data_small.drop_duplicates(subset='PATNO')

#filter for HYPOSMIA =1
#data_large = data_large[data_large['HYPOSMIA'] == 1]
#data_small = data_small[data_small['HYPOSMIA'] == 1]

# Add a target column to each dataset
data_large['target'] = 0
data_small['target'] = 1

# Combine the datasets
combined_data = pd.concat([data_large, data_small])

# Reset index of combined data
combined_data.reset_index(drop=True, inplace=True)

# Define the columns for stratification
age_var = 'AGE'
sex_var = 'BIRTHSEX'
target_var = 'target'

# Ensure both datasets have age_var, sex_var, and target_var columns
assert age_var in combined_data.columns and sex_var in combined_data.columns and target_var in combined_data.columns

# Calculate percentiles for the age variable
percentiles = np.percentile(combined_data[age_var], [20, 40, 60, 80])
labels = ['0-20th', '21-40th', '41-60th', '61-80th', '81-100th']
combined_data['age_binned'] = pd.cut(combined_data[age_var], bins=[combined_data[age_var].min()-1, *percentiles, combined_data[age_var].max()+1], labels=labels, include_lowest=True)

# Create a stratification key
combined_data['stratify_key'] = combined_data.apply(lambda row: f"{row['age_binned']}_{row[sex_var]}_{row[target_var]}", axis=1)

# Split the data into train and test sets
train, test = train_test_split(combined_data, test_size=0.2, stratify=combined_data['stratify_key'], random_state=42)

# Drop the stratification key and binned age column
train = train.drop(columns=['stratify_key', 'age_binned'])
test = test.drop(columns=['stratify_key', 'age_binned'])

#print mean, std and count of sex, age and target in train and test
print(train['BIRTHSEX'].mean(), test['BIRTHSEX'].mean())
print(train['BIRTHSEX'].std(), test['BIRTHSEX'].std())
print(train['BIRTHSEX'].count(), test['BIRTHSEX'].count())
print(train['AGE'].mean(), test['AGE'].mean())
print(train['AGE'].std(), test['AGE'].std())
print(train['AGE'].count(), test['AGE'].count())
print(train['target'].mean(), test['target'].mean())
print(train['target'].std(), test['target'].std())
print(train['target'].count(), test['target'].count())

#switch random patients from train to test until target is balanced
while abs(train['target'].mean() - test['target'].mean()) > 0.01:
    #get random patient from train
    random_patient = train.sample()
    #switch to test
    test = pd.concat([test, random_patient])
    train = train.drop(random_patient.index)
    #now switch from test to train
    random_patient = test.sample()
    train = pd.concat([train, random_patient])
    test = test.drop(random_patient.index)
    #print mean of target in train and test
    print(train['target'].mean(), test['target'].mean())
print ("Balanced target")
#print mean, std and count of sex, age and target in train and test
print(train['BIRTHSEX'].mean(), test['BIRTHSEX'].mean())
print(train['BIRTHSEX'].std(), test['BIRTHSEX'].std())
print(train['BIRTHSEX'].count(), test['BIRTHSEX'].count())
print(train['AGE'].mean(), test['AGE'].mean())
print(train['AGE'].std(), test['AGE'].std())
print(train['AGE'].count(), test['AGE'].count())
print(train['target'].mean(), test['target'].mean())
print(train['target'].std(), test['target'].std())
print(train['target'].count(), test['target'].count())


#print shapes
print(train.shape)
print(test.shape)

0.6335616438356164 0.636986301369863
0.4822442385632843 0.4825240674547979
584 146
68.72602739726027 68.79178082191783
5.818077403372382 6.0281238214557336
584 146
0.4965753424657534 0.5068493150684932
0.5004168942693765 0.501674098669036
584 146
0.4948630136986301 0.5136986301369864
0.4931506849315068 0.5205479452054794
0.4931506849315068 0.5205479452054794
0.4931506849315068 0.5205479452054794
0.4931506849315068 0.5205479452054794
0.4948630136986301 0.5136986301369864
0.4931506849315068 0.5205479452054794
0.4931506849315068 0.5205479452054794
0.4948630136986301 0.5136986301369864
0.4948630136986301 0.5136986301369864
0.4948630136986301 0.5136986301369864
0.4948630136986301 0.5136986301369864
0.4931506849315068 0.5205479452054794
0.4931506849315068 0.5205479452054794
0.4948630136986301 0.5136986301369864
0.4965753424657534 0.5068493150684932
0.4965753424657534 0.5068493150684932
0.4965753424657534 0.5068493150684932
0.4965753424657534 0.5068493150684932
0.4948630136986301 0.5136986301

In [28]:
#check if there is any duplicate in PATNO
print(train['PATNO'].duplicated().sum())
print(test['PATNO'].duplicated().sum())

print(train.shape)
print(test.shape)

train_columns = train.columns
test_columns = test.columns

if train_columns.equals(test_columns):
    print("Columns are the same")

for column in train_columns:
    print(column)

0
0
(584, 222)
(146, 222)
Columns are the same
PATNO
EVENT_ID
PAG_NAME
INFODT
HIQ1
HIQ2
HIQ3
HIQ4
HIQ5
HIQ6
ORIG_ENTRY
LAST_UPDATE
UPSITSNTDT
UPSITENTDT
SCRINTDT
SCRINTST
SCRTRDT
SCRSITE
SCRVISTYPE
SCRSCHEDDT
SCRVISST
SCRRSNUM
BLINTDT
BLINTST
BLTRDT
BLSITE
BLSCHEDDT
BLVISST
BLRSNUM
COHORT
TRACK
CONSENTDT
CONSENTST
ENROLLST
BIRTHSEX
RAASIAN
RABLACK
RADEC
RAHAWOPI
RAINDALS
RAUNK
RAWHITE
HISPLAT
COUNTRY
SCREENINGID
SCREENDT
SCREENTM
CAMPAIGN
REQAGE
REQLOC
DIAGPD
PDMED
DIAGAGE2
RTRNLTR
PARTNERID
USQ1
USQ2
USQ3
USQ4
USQ5
USQ6
USQ7
USQ8
USQ8A
USQ8B
USQ8C
USQ9
USQ10
REC_ID
SCENT_01_CORRECT
SCENT_01_RESPONSE
SCENT_02_CORRECT
SCENT_02_RESPONSE
SCENT_03_CORRECT
SCENT_03_RESPONSE
SCENT_04_CORRECT
SCENT_04_RESPONSE
SCENT_05_CORRECT
SCENT_05_RESPONSE
SCENT_06_CORRECT
SCENT_06_RESPONSE
SCENT_07_CORRECT
SCENT_07_RESPONSE
SCENT_08_CORRECT
SCENT_08_RESPONSE
SCENT_09_CORRECT
SCENT_09_RESPONSE
SCENT_10_CORRECT
SCENT_10_RESPONSE
SCENT_11_CORRECT
SCENT_11_RESPONSE
SCENT_12_CORRECT
SCENT_12_RESPONSE
SCENT_1

In [29]:
#sum all correct columns (that contain "CORRECT")

#print mean and std of total_correct
print(train['TOTAL_CORRECT'].mean(), test['TOTAL_CORRECT'].mean())
#MIN AND MAX
print(train['TOTAL_CORRECT'].min(), test['TOTAL_CORRECT'].min())
print(train['TOTAL_CORRECT'].max(), test['TOTAL_CORRECT'].max())
#print mean and std of TOTAL_CORRECT per target
print(train.groupby('target')['TOTAL_CORRECT'].mean())
print(test.groupby('target')['TOTAL_CORRECT'].mean())
#print mean and std of TOTAL_CORRECT
print(train.groupby('HYPOSMIA')['TOTAL_CORRECT'].min())
print(test.groupby('HYPOSMIA')['TOTAL_CORRECT'].min())
#now per HYPOSMIA
print(train.groupby('HYPOSMIA')['TOTAL_CORRECT'].max())
print(test.groupby('HYPOSMIA')['TOTAL_CORRECT'].max())
 


28.362585616438356 27.402397260273972
6.0 8.0
40.0 40.0
target
0    34.467577
1    22.215636
Name: TOTAL_CORRECT, dtype: float64
target
0    32.726027
1    22.078767
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
0.0    7.0
1.0    6.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
0.0    10.0
1.0     8.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
0.0    40.0
1.0    30.0
Name: TOTAL_CORRECT, dtype: float64
HYPOSMIA
0.0    40.0
1.0    28.0
Name: TOTAL_CORRECT, dtype: float64


In [30]:
# Save the train and test sets to new CSV files
train.to_csv('data/processed/train_data_all.csv', index=False)
test.to_csv('data/processed/test_data_all.csv', index=False)

print(f"Train set created with {len(train)} instances.")
print(f"Test set created with {len(test)} instances.")

Train set created with 584 instances.
Test set created with 146 instances.
